In [ ]:
# aWaifu - random waifu generator, run in Google Colab
# Aki Hakune, October 27th 2021

In [ ]:
# Turn these on if you are too lazy to read the code
autoRun = False
autoSave = False
verbose = False

In [ ]:
!pip install -q Waifulabs
import waifulabs
import cv2
import time
import os
import requests
import shutil
import json
import random
from typing import Dict
from google.colab.patches import cv2_imshow
from google.colab import files

In [ ]:
class Waifus:
  def __init__ (self, outputPath:str = 'waifus/',
                numberOfProfiles:int = 10,
                verbosity:bool = False,
                bigWaifu:bool = False,
                noProfile:bool = False,
                noImage:bool = False):
    self.dataPath = outputPath
    self.numberOfProfiles:int = numberOfProfiles
    self.verbose:bool = verbosity
    self.bigWaifu:bool = bigWaifu
    self.noProfile = noProfile
    self.noImage = noImage


  # Methods are arranged from more public -> more private ones
  # The last method is an exception, as it's the main method of this class


  @staticmethod
  def cleanUpPreviousRuns() -> None:
    """Delete data from previous executions"""
    defaultDataDir:str = 'waifus'
    shutil.rmtree(defaultDataDir, ignore_errors=True)


  @staticmethod
  def showWaifuImages() -> None:
    waifuImageDir:str = 'waifus'

    if os.path.isdir(waifuImageDir):
      for filename in os.listdir(waifuImageDir):
        if filename.startswith('waifu-') or filename.endswith('.png'):
          imagePath:str = waifuImageDir + '/' + filename
          print(f"Showing: {imagePath}\n")
          img = cv2.imread(imagePath, -1)
          cv2_imshow(img)
          print('\n\n\n')
          
    else:
      raise FileNotFoundError("Make sure that you've generated some profiles")

  
  @staticmethod
  def getAllInfo() -> None:
    """Download every generated information under zipped format"""
    try:
      shutil.make_archive('waifus', 'zip', 'waifus')
    except FileNotFoundError:
      print("Please make sure that you have generated 'waifus' directory first.")
      return None
      
    print('Zipping files... Please patiently wait...', end='\r')
    files.download('waifus.zip') # might not work in Firefox


  @staticmethod
  def getRandomAge() -> None:
    """Generate completely random, unrelated age"""
    return random.choice([
        random.randint(3, 25),        # age of human waifu
        random.randint(10**3, 10**5)  # age of non-human waifu
    ])


  @staticmethod
  def getRandomRace(age:int) -> str:
    """Randomizing a race"""
    # TODO: Add more races
    # TODO: Load from external sources
    humanHighestAge:int = 25
    NON_HUMAN_RACES = ['Dark Hagravens', 'Demi Fiends', 'Lost Nymphs', 'Forlorn Hags', 'Highborn Fiends', 'Elemental Hydra', 'Bog Nymphs', 'Shard Undine', 'Velvet People', 'Western Fairies', 'Forest Elf', 'Frost Dwarf', 'Demon', 'Dragonman', 'Fairy', 'Fire Soul', 'Half-Elf', 'High Elf', 'Lamia', 'Moon Elf', 'Night Elf']

    if age > humanHighestAge:
      return random.choice(NON_HUMAN_RACES)
    return random.choice(['Human', random.choice(NON_HUMAN_RACES)])


  def _vbose(self, contextType:str, context) -> None:
    """Logging, verbose messages and image showing"""
    if self.verbose:
      if contextType == 'text':
        print(context, end='\n')
      elif contextType == 'image':
        img = cv2.imread(context, -1)
        cv2_imshow(img)
        print('\n\n\n')
      elif contextType == 'dictionary':
        print(json.dumps(context, indent=4, ensure_ascii=False))
        print()
      else:
        print(f"Unknown type logging: {context}")


  def _getRandomProfile(self, imagePath:str) -> 'json data':
    """Generate random profile"""

    profileDataPath:str = self.dataPath + 'profile.json'

    # Using free, no-authentication Name Fake API
    apiHost:str = 'https://api.namefake.com'
    endPoint:str = apiHost + '/random/female'

    call = requests.get(endPoint)
    rawData = call.json()

    # Because randomizing race depends on life span
    waifuAge = self.getRandomAge()
    waifuRace = self.getRandomRace(waifuAge)

    waifuData:Dict[str:str] = {
        'image': imagePath,
        'name': rawData['name'],
        'code_name': rawData['email_u'],
        'age': waifuAge,
        'race': waifuRace,
        'current_location': rawData['address'].replace('\n', ' '),
        'birthday': rawData['birth_data'][5:],
        'representative_color': rawData['color'],
        'blood_type': rawData['blood'],
    }

    self._vbose('dictionary', waifuData)

    with open(profileDataPath, 'a+') as f:
      f.write(json.dumps(waifuData, indent=4, ensure_ascii=False))
      f.write('\n\n\n')



  def _getRandomImages(self, filename:str) -> None:
    """Getting waifu images from waifulabs"""
    if self.bigWaifu:
      waifu = waifulabs.GenerateWaifu().GenerateBigWaifu()
    else:
      waifu = waifulabs.GenerateWaifu()

    waifu.save(self.dataPath + filename)
    self._vbose('image', self.dataPath + filename)


  def generateProfiles(self) -> None:
    """Generate full waifu profiles"""

    # Set up data directory the first run
    if not os.path.isdir(self.dataPath):
      os.mkdir(self.dataPath)

    for i in range(self.numberOfProfiles):
      self._vbose('text', f'ID: {i + 1}/{self.numberOfProfiles}\n')

      if not self.noProfile:
        self._getRandomProfile(imagePath = self.dataPath + f"waifu-{i + 1}.png")
      if not self.noImage:
        self._getRandomImages(filename=f"waifu-{i + 1}.png")

      time.sleep(1) # try not to DOS Waifulab's servers

In [ ]:
if autoRun:
  waifuGen = Waifus(verbosity=verbose)
  waifuGen.generateProfiles()
  if autoSave:
    waifuGen.getAllInfo()